<a href="https://colab.research.google.com/github/vamsijan/audio_ravdess/blob/main/MultiModal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**create data**

In [ ]:
import numpy as np, pandas as pd
from collections import defaultdict
import pickle
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()


np.random.seed(17)

pre_data = np.asarray(pd.read_csv("/content/drive/MyDrive/contextual-multimodal-fusion/data/transcripts.csv" , header=None))

train = pd.read_csv("/content/drive/MyDrive/contextual-multimodal-fusion/data/text_train.csv", header=None)
test = pd.read_csv("/content/drive/MyDrive/contextual-multimodal-fusion/data/text_test.csv", header=None)
train = np.asarray(train)
test = np.asarray(test)
train_index = np.asarray(train[:,0], dtype = 'int')
test_index = np.asarray(test[:,0], dtype = 'int')



def main(name):

    path = "/content/drive/MyDrive/contextual-multimodal-fusion/data/"+name+"/"+name
    print (path)
    train_video_mapping=defaultdict(list)
    train_video_mapping_index=defaultdict(list)
    test_video_mapping=defaultdict(list)
    test_video_mapping_index=defaultdict(list)

    data_train = np.asarray(pd.read_csv(path+"_train0.csv", header=None))
    data_test = np.asarray(pd.read_csv(path+"_test0.csv", header=None))

    for i in range(train_index.shape[0]):
        train_video_mapping[pre_data[train_index[i]][0].rsplit("_",1)[0] ].append(train_index[i])
        train_video_mapping_index[pre_data[train_index[i]][0].rsplit("_",1)[0] ].append( int(pre_data[train_index[i]][0].rsplit("_",1)[1]) )

    for i in range(test_index.shape[0]):
        test_video_mapping[pre_data[test_index[i]][0].rsplit("_",1)[0] ].append(test_index[i])
        test_video_mapping_index[pre_data[test_index[i]][0].rsplit("_",1)[0] ].append( int(pre_data[test_index[i]][0].rsplit("_",1)[1]) )

    train_indices = dict((c, i) for i, c in enumerate(train_index))
    test_indices = dict((c, i) for i, c in enumerate(test_index))

    max_len = 0
    for key,value in train_video_mapping.items():
        max_len = max(max_len , len(value))
    for key,value in test_video_mapping.items():
        max_len = max(max_len, len(value))

    pad = np.asarray([0 for i in range(data_train[0][:-1].shape[0])])

    print ("Mapping train")

    train_data_X =[]
    train_data_Y =[]
    train_length =[]
    for key,value in train_video_mapping.items():

        
        lst = np.column_stack((train_video_mapping_index[key],value)  )
        ind = np.asarray(sorted(lst,key=lambda x: x[0]))


        lst_X, lst_Y=[],[]
        ctr=0;
        for i in range(ind.shape[0]):
            ctr+=1
            #lst_X.append(preprocessing.scale( min_max_scaler.fit_transform(data_train[train_indices[ind[i,1]]][:-1])))
            lst_X.append(data_train[train_indices[ind[i,1]]][:-1])
            lst_Y.append(data_train[train_indices[ind[i,1]]][-1])
        train_length.append(ctr)
        for i in range(ctr, max_len):
            lst_X.append(pad)
            lst_Y.append(0) #dummy label
        
        train_data_X.append(lst_X)
        train_data_Y.append(lst_Y)
    

    test_data_X =[]
    test_data_Y =[]
    test_length =[]

    print ("Mapping test")

    for key,value in test_video_mapping.items():

        lst = np.column_stack((test_video_mapping_index[key],value)  )
        ind = np.asarray(sorted(lst,key=lambda x: x[0]))

        lst_X, lst_Y=[],[]
        ctr=0
        for i in range(ind.shape[0]):
            ctr+=1
            #lst_X.append(preprocessing.scale( min_max_scaler.transform(data_test[test_indices[ind[i,1]]][:-1])))
            lst_X.append(data_test[test_indices[ind[i,1]]][:-1])
            lst_Y.append(data_test[test_indices[ind[i,1]]][-1])
        test_length.append(ctr)
        for i in range(ctr, max_len):
            lst_X.append(pad)
            lst_Y.append(0) #dummy label

        test_data_X.append(np.asarray(lst_X))
        test_data_Y.append(np.asarray(lst_Y))

    train_data_X = np.asarray(train_data_X)
    test_data_X = np.asarray(test_data_X)
    print (train_data_X.shape, test_data_X.shape,len(train_length), len(test_length))

    print ("Dumping data")
    with open('/content/drive/MyDrive/contextual-multimodal-fusion/input/'+name+'.pickle', 'wb') as handle:
        pickle.dump((train_data_X,  np.asarray(train_data_Y), test_data_X, np.asarray(test_data_Y), max_len ,train_length, test_length), handle, protocol=pickle.HIGHEST_PROTOCOL)



    


if __name__ == "__main__":

    names = ['text','audio','video']
    for nm in names:
        main(nm)

/content/drive/MyDrive/contextual-multimodal-fusion/data/text/text
Mapping train
Mapping test
(62, 63, 100) (31, 63, 100) 62 31
Dumping data
/content/drive/MyDrive/contextual-multimodal-fusion/data/audio/audio
Mapping train
Mapping test
(62, 63, 73) (31, 63, 73) 62 31
Dumping data
/content/drive/MyDrive/contextual-multimodal-fusion/data/video/video
Mapping train
Mapping test
(62, 63, 100) (31, 63, 100) 62 31
Dumping data


**trimodal_attention_model**

In [ ]:
import gc, numpy as np, pickle
import tensorflow as tf
from keras.models import Model
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input, Bidirectional, GRU, Masking, Dense, Dropout, TimeDistributed, Lambda, Activation, dot, multiply, concatenate
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)


def calc_test_result(result, test_label, test_mask, print_detailed_results=False):
    '''
    # Arguments
        predicted test labels, gold test labels and test mask

    # Returns
        accuracy of the predicted labels
    '''
    true_label=[]
    predicted_label=[]

    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if test_mask[i,j]==1:
                true_label.append(np.argmax(test_label[i,j] ))
                predicted_label.append(np.argmax(result[i,j] ))
        
    if print_detailed_results:
        print ("Confusion Matrix :") 
        print (confusion_matrix(true_label, predicted_label))
        print ("Classification Report :") 
        print (classification_report(true_label, predicted_label))
    print ("Accuracy ", accuracy_score(true_label, predicted_label))
    return accuracy_score(true_label, predicted_label)



def create_one_hot_labels(train_label, test_label):
    '''
    # Arguments
        train and test labels (2D matrices)

    # Returns
        one hot encoded train and test labels (3D matrices)
    '''

    maxlen = int(max(train_label.max(), test_label.max()))
    
    train = np.zeros((train_label.shape[0], train_label.shape[1], maxlen+1))
    test = np.zeros((test_label.shape[0], test_label.shape[1], maxlen+1))
    
    for i in range(train_label.shape[0]):
        for j in range(train_label.shape[1]):
            train[i,j,train_label[i,j]] = 1

    for i in range(test_label.shape[0]):
        for j in range(test_label.shape[1]):
            test[i,j,test_label[i,j]] = 1

    return train, test


def create_mask(train_data, test_data, train_length, test_length):
    '''
    # Arguments
        train, test data (any one modality (text, audio or video)), utterance lengths in train, test videos

    # Returns
        mask for train and test data
    '''

    train_mask = np.zeros((train_data.shape[0], train_data.shape[1]), dtype='float')
    for i in range(len(train_length)):
        train_mask[i, :train_length[i]] = 1.0

    test_mask = np.zeros((test_data.shape[0], test_data.shape[1]), dtype='float')
    for i in range(len(test_length)):
        test_mask[i, :test_length[i]] = 1.0
    
    return train_mask, test_mask



(train_text, train_label, test_text, test_label, max_utt_len, train_len, test_len) = pickle.load(open('/content/drive/MyDrive/contextual-multimodal-fusion/input/text.pickle', 'rb'))
(train_audio, _, test_audio, _, _, _, _) = pickle.load(open('/content/drive/MyDrive/contextual-multimodal-fusion/input/audio.pickle', 'rb'))
(train_video, _, test_video, _, _, _, _) = pickle.load(open('/content/drive/MyDrive/contextual-multimodal-fusion/input/video.pickle', 'rb'))

train_label, test_label = create_one_hot_labels(train_label.astype('int'), test_label.astype('int'))

train_mask, test_mask = create_mask(train_text, test_text, train_len, test_len)


num_train = int(len(train_text)*0.8)

train_text, dev_text = train_text[:num_train, :, :], train_text[num_train:, :, :]
train_audio, dev_audio = train_audio[:num_train, :, :], train_audio[num_train:, :, :]
train_video, dev_video = train_video[:num_train, :, :], train_video[num_train:, :, :]
train_label, dev_label = train_label[:num_train, :, :], train_label[num_train:, :, :]
train_mask, dev_mask = train_mask[:num_train, :], train_mask[num_train:, :]



def bi_modal_attention(x, y):
    
    ''' 
    .  stands for dot product 
    *  stands for elemwise multiplication
    {} stands for concatenation
        
    m1 = x . transpose(y) ||  m2 = y . transpose(x) 
    n1 = softmax(m1)      ||  n2 = softmax(m2)
    o1 = n1 . y           ||  o2 = m2 . x
    a1 = o1 * x           ||  a2 = o2 * y
       
    return {a1, a2}
        
    '''
     
    m1 = dot([x, y], axes=[2, 2])
    n1 = Activation('softmax')(m1)
    o1 = dot([n1, y], axes=[2, 1])
    a1 = multiply([o1, x])

    m2 = dot([y, x], axes=[2, 2])
    n2 = Activation('softmax')(m2)
    o2 = dot([n2, x], axes=[2, 1])
    a2 = multiply([o2, y])

    return concatenate([a1, a2])


def self_attention(x):
    
    ''' 
    .  stands for dot product 
    *  stands for elemwise multiplication
        
    m = x . transpose(x)
    n = softmax(m)
    o = n . x  
    a = o * x           
       
    return a
        
    '''

    m = dot([x, x], axes=[2,2])
    n = Activation('softmax')(m)
    o = dot([n, x], axes=[2,1])
    a = multiply([o, x])
        
    return a
    

def contextual_attention_model(mode):
    
    ########### Input Layer ############
        
    in_text = Input(shape=(train_text.shape[1], train_text.shape[2]))
    in_audio = Input(shape=(train_audio.shape[1], train_audio.shape[2]))
    in_video = Input(shape=(train_video.shape[1], train_video.shape[2]))
        
        
    ########### Masking Layer ############
        
    masked_text = Masking(mask_value=0)(in_text)
    masked_audio = Masking(mask_value=0)(in_audio)
    masked_video = Masking(mask_value=0)(in_video)
            
            
    ########### Recurrent Layer ############            
        
    drop_rnn = 0.7
    gru_units = 300
            
    rnn_text = Bidirectional(GRU(gru_units, return_sequences=True, dropout=0.5, recurrent_dropout=0.5), merge_mode='concat')(masked_text)
    rnn_audio = Bidirectional(GRU(gru_units, return_sequences=True, dropout=0.5, recurrent_dropout=0.5), merge_mode='concat')(masked_audio)
    rnn_video = Bidirectional(GRU(gru_units, return_sequences=True, dropout=0.5, recurrent_dropout=0.5), merge_mode='concat')(masked_video)        
            
    rnn_text = Dropout(drop_rnn)(rnn_text)
    rnn_audio = Dropout(drop_rnn)(rnn_audio)
    rnn_video = Dropout(drop_rnn)(rnn_video)
            
        
    ########### Time-Distributed Dense Layer ############
        
    drop_dense = 0.7
    dense_units = 100
            
    dense_text = Dropout(drop_dense)(TimeDistributed(Dense(dense_units, activation='tanh'))(rnn_text))
    dense_audio = Dropout(drop_dense)(TimeDistributed(Dense(dense_units, activation='tanh'))(rnn_audio))
    dense_video = Dropout(drop_dense)(TimeDistributed(Dense(dense_units, activation='tanh'))(rnn_video))
                      
        
    ########### Attention Layer ############
        
    ## Multi Modal Multi Utterance Bi-Modal attention ##
    if mode == 'MMMU_BA':
            
        vt_att = bi_modal_attention(dense_video, dense_text)
        av_att = bi_modal_attention(dense_audio, dense_video)
        ta_att = bi_modal_attention(dense_text, dense_audio)
                
        merged = concatenate([vt_att, av_att, ta_att, dense_video, dense_audio, dense_text])
                 
        
    ## Multi Modal Uni Utterance Self Attention ##
    elif mode == 'MMUU_SA':
            
        attention_features = []
            
        for k in range(max_utt_len):
                
            # extract multi modal features for each utterance #
            m1 = Lambda(lambda x: x[:, k:k+1, :])(dense_video)
            m2 = Lambda(lambda x: x[:, k:k+1, :])(dense_audio)
            m3 = Lambda(lambda x: x[:, k:k+1, :])(dense_text)
                
            utterance_features = concatenate([m1, m2, m3], axis=1)
            attention_features.append(self_attention(utterance_features))

        merged_attention = concatenate(attention_features, axis=1)
        merged_attention = Lambda(lambda x: K.reshape(x, (-1, max_utt_len, 3*dense_units)))(merged_attention)
            
        merged = concatenate([merged_attention, dense_video, dense_audio, dense_text])
            
        
    ## Multi Utterance Self Attention ##
    elif mode == 'MU_SA':
            
        vv_att = self_attention(dense_video)
        tt_att = self_attention(dense_text)
        aa_att = self_attention(dense_audio)
            
        merged = concatenate([aa_att, vv_att, tt_att, dense_video, dense_audio, dense_text])
            
        
    ## No Attention ##    
    elif mode == 'None':
            
        merged = concatenate([dense_video, dense_audio, dense_text])
                
    else:
        print ("Mode must be one of 'MMMU-BA', 'MMUU-SA', 'MU-SA' or 'None'.")
        return
                
        
    ########### Output Layer ############
        
    output = TimeDistributed(Dense(2, activation='softmax'))(merged)
    model = Model([in_text, in_audio, in_video], output)        
        
    return model



def train(mode):
    
    runs = 5
    accuracy = []
    
    for j in range(runs):
        
        np.random.seed(j)
        tf.compat.v1.set_random_seed(j)
        
        # compile model #
        model = contextual_attention_model(mode)
        model.compile(optimizer='adam', loss='categorical_crossentropy', weighted_metrics=[], sample_weight_mode='temporal', metrics=['accuracy'])
        
        # set callbacks #   
        path = '/content/drive/MyDrive/contextual-multimodal-fusion/weights/Mosi_Trimodal_' + mode + '_Run_' + str(j) + '.hdf5'

        early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0)
        check = ModelCheckpoint(path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=0)

        
        # train model #
        history = model.fit([train_text, train_audio, train_video], train_label,
                            epochs=15,
                            batch_size=32,
                            sample_weight=train_mask,
                            shuffle=True, 
                            callbacks=[early_stop, check],
                            # validation_data=([dev_text, dev_audio, dev_video], dev_label, dev_mask),
                            validation_data=([test_text, test_audio, test_video], test_label, test_mask),
                            verbose=1)
                    
            
        # test results #
        model.load_weights(path)
        test_predictions = model.predict([test_text, test_audio, test_video])
        test_accuracy = calc_test_result(test_predictions, test_label, test_mask)
        accuracy.append(test_accuracy)

        
        # release gpu memory #
        K.clear_session()
        del model, history
        gc.collect()
        
        
    # summarize test results #

    avg_accuracy = sum(accuracy)/len(accuracy)
    max_accuracy = max(accuracy)
    
    print ('Mode: ', mode)
    print ('Avg Test Accuracy:', '{0:.4f}'.format(avg_accuracy), '|| Max Test Accuracy:', '{0:.4f}'.format(max_accuracy))
    print ('-'*55)


if __name__=="__main__":
    
    for mode in ['MMMU_BA', 'MMUU_SA', 'MU_SA', 'None']:
        train(mode)

Epoch 1/15
2/2 [==============================] - 28s 7s/step - loss: 0.4608 - accuracy: 0.5275 - val_loss: 0.2342 - val_accuracy: 0.7008
Epoch 2/15
2/2 [==============================] - 7s 4s/step - loss: 0.3422 - accuracy: 0.6203 - val_loss: 0.2104 - val_accuracy: 0.7301
Epoch 3/15
2/2 [==============================] - 7s 4s/step - loss: 0.2707 - accuracy: 0.6813 - val_loss: 0.1896 - val_accuracy: 0.7766
Epoch 4/15
2/2 [==============================] - 9s 5s/step - loss: 0.2319 - accuracy: 0.7440 - val_loss: 0.1769 - val_accuracy: 0.7899
Epoch 5/15
2/2 [==============================] - 7s 4s/step - loss: 0.1919 - accuracy: 0.7869 - val_loss: 0.1720 - val_accuracy: 0.8005
Epoch 6/15
2/2 [==============================] - 7s 3s/step - loss: 0.1460 - accuracy: 0.8591 - val_loss: 0.1712 - val_accuracy: 0.7992
Epoch 7/15
2/2 [==============================] - 7s 3s/step - loss: 0.1254 - accuracy: 0.8582 - val_loss: 0.1740 - val_accuracy: 0.7992
Epoch 8/15
2/2 [========================

1/1 [==============================] - 3s 3s/step
Accuracy  0.8018617021276596
Mode:  MMMU_BA
Avg Test Accuracy: 0.8048 || Max Test Accuracy: 0.8098
-------------------------------------------------------
Epoch 1/15
2/2 [==============================] - 41s 8s/step - loss: 0.3185 - accuracy: 0.5670 - val_loss: 0.2340 - val_accuracy: 0.6396
Epoch 2/15
2/2 [==============================] - 8s 4s/step - loss: 0.1683 - accuracy: 0.7869 - val_loss: 0.1876 - val_accuracy: 0.7580
Epoch 3/15
2/2 [==============================] - 8s 4s/step - loss: 0.0999 - accuracy: 0.8935 - val_loss: 0.1851 - val_accuracy: 0.7726
Epoch 4/15
2/2 [==============================] - 8s 4s/step - loss: 0.0740 - accuracy: 0.9261 - val_loss: 0.1960 - val_accuracy: 0.7739
Epoch 5/15
2/2 [==============================] - 8s 4s/step - loss: 0.0479 - accuracy: 0.9536 - val_loss: 0.2152 - val_accuracy: 0.7832
Epoch 6/15
2/2 [==============================] - 7s 4s/step - loss: 0.0308 - accuracy: 0.9742 - val_loss: 0.

1/1 [==============================] - 4s 4s/step
Accuracy  0.7832446808510638
Epoch 1/15
2/2 [==============================] - 41s 8s/step - loss: 0.2858 - accuracy: 0.6151 - val_loss: 0.1846 - val_accuracy: 0.7753
Epoch 2/15
2/2 [==============================] - 7s 4s/step - loss: 0.1407 - accuracy: 0.8368 - val_loss: 0.1775 - val_accuracy: 0.7912
Epoch 3/15
2/2 [==============================] - 7s 4s/step - loss: 0.0909 - accuracy: 0.9064 - val_loss: 0.1845 - val_accuracy: 0.7965
Epoch 4/15
2/2 [==============================] - 8s 4s/step - loss: 0.0619 - accuracy: 0.9356 - val_loss: 0.1908 - val_accuracy: 0.8032
Epoch 5/15
2/2 [==============================] - 7s 3s/step - loss: 0.0411 - accuracy: 0.9605 - val_loss: 0.2064 - val_accuracy: 0.8032
Epoch 6/15
2/2 [==============================] - 7s 4s/step - loss: 0.0300 - accuracy: 0.9759 - val_loss: 0.2257 - val_accuracy: 0.8019
Epoch 7/15
2/2 [==============================] - 8s 4s/step - loss: 0.0239 - accuracy: 0.9811 - v